## Model Training

#### Importing Data and Necessary Libraries



In [31]:
!pip install catboost 



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached catboost-1.2.5-cp311-cp311-win_amd64.whl.metadata (1.2 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/101.1 MB 245.8 kB/s eta 0:06:52
   ---------------------------------------- 0.1/101.1 MB 291.5 kB/s eta 0:05:47
   ---------------------------------------- 0.1/101.1 MB 291.5 kB/s eta 0:05:47
   ---------------------------------------- 0.1/101.1 MB 291.5 kB/s eta 0:05:47
   ---------------------------------------- 0.2/101.1 MB 286.7 kB/s eta 0:05:53
   ---------------------------------------- 0.2/101.1 MB 286.7 kB/s eta 0:05:53
   ----------------------

In [36]:
!pip install xgboost


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached xgboost-2.1.0-py3-none-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/124.9 MB 210.1 kB/s eta 0:09:55
   ---------------------------------------- 0.1/124.9 MB 210.1 kB/s eta 0:09:55
   ---------------------------------------- 0.1/124.9 MB 210.1 kB/s eta 0:09:55
   ---------------------------------------- 0.1/12

In [39]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings


### Importing dataset

In [21]:
df = pd.read_csv('data/cleaned_dataset.csv')
data = df.copy()

In [22]:
df.shape

(23469, 24)

In [23]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,VIN,drive,type,paint_color,image_url,description,state,lat,long,posting_date
0,7315233705,https://omaha.craigslist.org/ctd/d/fremont-201...,omaha / council bluffs,https://omaha.craigslist.org,36995,2013.0,ram,2500,good,6 cylinders,...,3C6UR5DL5DG584817,4wd,pickup,black,https://images.craigslist.org/00o0o_bp0dB4eW2i...,2013 Ram 2500 Crew Cab Big Horn Pickup 4D 6 1/...,ia,41.424186,-96.499908,2021-05-01T07:32:52-0500
1,7306636389,https://tricities.craigslist.org/cto/d/bristol...,tri-cities,https://tricities.craigslist.org,10000,2011.0,bmw,550i,good,6 cylinders,...,1FMJU1JT1HEA52352,4wd,sedan,grey,https://images.craigslist.org/00w0w_53YhtQfYp5...,This car is in good condition. Runs and drives...,tn,36.568600,-82.181900,2021-04-14T10:12:48-0400
2,7316498014,https://greenville.craigslist.org/ctd/d/greenv...,greenville / upstate,https://greenville.craigslist.org,34990,2018.0,jeep,grand cherokee trailhawk,good,6 cylinders,...,1C4RJFLG1JC138453,4wd,other,black,https://images.craigslist.org/00202_7MHlVPbgCs...,Carvana is the safer way to buy a car During t...,sc,34.830000,-82.370000,2021-05-03T19:22:00-0400
3,7311169521,https://oklahomacity.craigslist.org/cto/d/hebe...,oklahoma city,https://oklahomacity.craigslist.org,6000,1964.0,chevrolet,shortbed,good,6 cylinders,...,1FMJU1JT1HEA52352,4wd,sedan,white,https://images.craigslist.org/00a0a_4W3LC9j87R...,1964 Chevy short bed step side rolling body. ...,ok,35.510300,-92.039200,2021-04-22T20:24:36-0500
4,7310457441,https://yakima.craigslist.org/ctd/d/kittitas-2...,yakima,https://yakima.craigslist.org,12988,2007.0,toyota,highlander,excellent,6 cylinders,...,1FMJU1JT1HEA52352,4wd,sedan,white,https://images.craigslist.org/00f0f_iKWWuC1aUz...,ＷＩＮＤＹ ＣＨＥＶＲＯＬＥＴ ⭐ No Payments For 90 Days* ⭐...,wa,46.999600,-120.516300,2021-04-21T11:26:09-0700


In [24]:
columns_to_drop = ['url','id', 'region_url', 'VIN', 'image_url', 'description', 'state', 'region', 'posting_date']

# Drop columns from df
df = df.drop(columns=columns_to_drop, axis=1)

In [25]:
df

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,lat,long
0,36995,2013.0,ram,2500,good,6 cylinders,diesel,133863.0,clean,automatic,4wd,pickup,black,41.424186,-96.499908
1,10000,2011.0,bmw,550i,good,6 cylinders,gas,112000.0,clean,manual,4wd,sedan,grey,36.568600,-82.181900
2,34990,2018.0,jeep,grand cherokee trailhawk,good,6 cylinders,gas,37509.0,clean,other,4wd,other,black,34.830000,-82.370000
3,6000,1964.0,chevrolet,shortbed,good,6 cylinders,gas,72812.0,missing,manual,4wd,sedan,white,35.510300,-92.039200
4,12988,2007.0,toyota,highlander,excellent,6 cylinders,gas,91510.0,clean,automatic,4wd,sedan,white,46.999600,-120.516300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23464,2990,2004.0,chevrolet,tahoe ls,good,8 cylinders,gas,254000.0,clean,automatic,4wd,SUV,black,45.585100,-93.596100
23465,22500,2002.0,toyota,tacoma xtracab,good,6 cylinders,gas,98368.0,clean,manual,4wd,pickup,custom,44.922500,-123.320000
23466,9997,2011.0,mercedes-benz,c-class,excellent,6 cylinders,gas,109646.0,clean,automatic,rwd,sedan,black,36.062162,-95.797464
23467,16590,2017.0,ford,taurus se sedan 4d,good,6 cylinders,gas,35599.0,clean,other,fwd,sedan,white,33.779214,-84.411811


### Preparing X and Y Variables

In [26]:
X = df.drop(columns=['price'],axis=1)

In [27]:
X.shape

(23469, 14)

In [28]:
y = df['price']

In [30]:
y

0        36995
1        10000
2        34990
3         6000
4        12988
         ...  
23464     2990
23465    22500
23466     9997
23467    16590
23468    20590
Name: price, Length: 23469, dtype: int64

### Column Transformation

In [33]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

In [34]:
num_features

Index(['year', 'odometer', 'lat', 'long'], dtype='object')

In [35]:
cat_features

Index(['manufacturer', 'model', 'condition', 'cylinders', 'fuel',
       'title_status', 'transmission', 'drive', 'type', 'paint_color'],
      dtype='object')

#### Importing necessary Libraries for Preprocessing and carrying out Preprocessing

In [38]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [44]:
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(handle_unknown='ignore')

In [45]:
# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)

In [46]:
X = preprocessor.fit_transform(X)

In [48]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape


((18775, 5414), (4694, 5414))

### Creating an Evaluation Function to give metrics after model training

In [50]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 10825.3000
- Mean Absolute Error: 4691.2827
- R2 Score: 0.6610
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 21488.7387
- Mean Absolute Error: 7540.0823
- R2 Score: -1.1609


Lasso
Model performance for Training set
- Root Mean Squared Error: 11872.8959
- Mean Absolute Error: 6559.6970
- R2 Score: 0.5923
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 20848.1807
- Mean Absolute Error: 7416.6607
- R2 Score: -1.0340


Ridge
Model performance for Training set
- Root Mean Squared Error: 12200.5467
- Mean Absolute Error: 5878.0181
- R2 Score: 0.5695
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 14227.2001
- Mean Absolute Error: 7136.6424
- R2 Score: 0.0528


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 13366.4470
- Mean Absolute Error: 5